In [1]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib 
from dateutil.relativedelta import relativedelta

from Tools import *
from Tools2 import *
import numpy as np

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler  
yf.pdr_override()

In [2]:
start="2020-12-31"
end="2022-12-31"
name="AAPL"

#MACD619RSI21_8020
stock_data=pct_change(name,start,end)
#stock_data=data.get_data_yahoo(tickers=name,start=start,end=end,progress=False)
    
stock_data=MACD_short(stock_data)
stock_data=RSI(stock_data)
stock_data=RSI_MACD(stock_data,21,80,20)

#stock_data["log_vaolume"] = np.log(stock_data['Volume'])

stock_data = stock_data.dropna()
del stock_data["real_situation"]
del stock_data["Position"]
del stock_data["Close_price_percentage"]


X = stock_data.iloc[:,:12]


st_x= StandardScaler()  
X= st_x.fit_transform(X)  

y = np.where(stock_data['Close'].shift(-1) > stock_data['Close'],1,-1)



In [3]:
split = int(0.7*len(stock_data))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]



In [4]:
model = LogisticRegression(max_iter=10000)
model = model.fit (X_train,y_train)

In [5]:
#pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

In [6]:
probability = model.predict_proba(X_test)
print(probability)

[[0.47407004 0.52592996]
 [0.46853363 0.53146637]
 [0.48746613 0.51253387]
 [0.47742063 0.52257937]
 [0.43009571 0.56990429]
 [0.3905986  0.6094014 ]
 [0.42688173 0.57311827]
 [0.46022302 0.53977698]
 [0.42837524 0.57162476]
 [0.43296459 0.56703541]
 [0.46925088 0.53074912]
 [0.46894212 0.53105788]
 [0.48038496 0.51961504]
 [0.49217721 0.50782279]
 [0.49445767 0.50554233]
 [0.48579141 0.51420859]
 [0.45868215 0.54131785]
 [0.45575069 0.54424931]
 [0.45453021 0.54546979]
 [0.4704731  0.5295269 ]
 [0.48274677 0.51725323]
 [0.49386508 0.50613492]
 [0.489909   0.510091  ]
 [0.48508988 0.51491012]
 [0.48574873 0.51425127]
 [0.46815668 0.53184332]
 [0.48047663 0.51952337]
 [0.50509149 0.49490851]
 [0.51266721 0.48733279]
 [0.50066062 0.49933938]
 [0.51377439 0.48622561]
 [0.53027613 0.46972387]
 [0.52061447 0.47938553]
 [0.50834417 0.49165583]
 [0.49853386 0.50146614]
 [0.51673278 0.48326722]
 [0.52922313 0.47077687]
 [0.56165758 0.43834242]
 [0.5357229  0.4642771 ]
 [0.52161778 0.47838222]


In [7]:
probability = model.predict_proba(X_test)
print(probability)

predicted = model.predict(X_test)
print(metrics.confusion_matrix(y_test, predicted))  
    
    



[[0.47407004 0.52592996]
 [0.46853363 0.53146637]
 [0.48746613 0.51253387]
 [0.47742063 0.52257937]
 [0.43009571 0.56990429]
 [0.3905986  0.6094014 ]
 [0.42688173 0.57311827]
 [0.46022302 0.53977698]
 [0.42837524 0.57162476]
 [0.43296459 0.56703541]
 [0.46925088 0.53074912]
 [0.46894212 0.53105788]
 [0.48038496 0.51961504]
 [0.49217721 0.50782279]
 [0.49445767 0.50554233]
 [0.48579141 0.51420859]
 [0.45868215 0.54131785]
 [0.45575069 0.54424931]
 [0.45453021 0.54546979]
 [0.4704731  0.5295269 ]
 [0.48274677 0.51725323]
 [0.49386508 0.50613492]
 [0.489909   0.510091  ]
 [0.48508988 0.51491012]
 [0.48574873 0.51425127]
 [0.46815668 0.53184332]
 [0.48047663 0.51952337]
 [0.50509149 0.49490851]
 [0.51266721 0.48733279]
 [0.50066062 0.49933938]
 [0.51377439 0.48622561]
 [0.53027613 0.46972387]
 [0.52061447 0.47938553]
 [0.50834417 0.49165583]
 [0.49853386 0.50146614]
 [0.51673278 0.48326722]
 [0.52922313 0.47077687]
 [0.56165758 0.43834242]
 [0.5357229  0.4642771 ]
 [0.52161778 0.47838222]


In [8]:
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

          -1       0.63      0.49      0.55        77
           1       0.54      0.68      0.60        68

    accuracy                           0.58       145
   macro avg       0.59      0.58      0.58       145
weighted avg       0.59      0.58      0.58       145



In [9]:
print(model.score(X_test,y_test))


0.5793103448275863
